# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import sys
import json

import nbformat
import nbconvert

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import mbrain as mb

# Development

In [4]:
with open('Example.ipynb') as f:
    # nb = json.load(f)
    nb = nbformat.read(f, as_version=4)

In [ ]:
nb

In [6]:
cell = nb['cells'][2].copy()

In [7]:
mb.utils.is_flashcard(nb['cells'][2])

True

In [8]:
source = cell.source

In [77]:
# This regex matches pattern:   <!--some text here-->
#
#                  v----------- pattern:   <!--
#                  | v--------- dot: matches any character except line break
#                  | |v-------- star: match zero or more of preceeding token
#                  | ||v------- lazy: make preceeding quantifier match as few characters as possible
#                vvvv|||vvv---- pattern:   -->
pattern_meta = r'<!--.*?-->'

def find_meta(string):
    meta = re.findall(pattern_meta, string)
    return meta[0]

def remove_meta(string):
    new_string = re.sub(pattern_meta, '', string)
    return new_string

In [78]:
# This regex matches pattern:   **some text here**
#
#                  v----------- pattern:   ** 
#                  | v--------- dot: matches any character except line break
#                  | |v-------- star: match zero or more of preceeding token
#                  | ||v------- lazy: make preceeding quantifier match as few characters as possible
#                vvvv|||vvvv--- pattern:   **
pattern_head = r'\*\*.*?\*\*'

def find_head(string):
    meta = re.findall(pattern_head, string)
    return meta[0]

def remove_head(string):
    new_string = re.sub(pattern_head, '', string)
    return new_string

In [10]:
meta = find_meta(source)
meta

'<!---->'

In [11]:
source = remove_meta(source)

In [12]:
head = find_head(source)

In [13]:
source = remove_head(source)

In [14]:
source = source.strip()

In [15]:
print(source)

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs \\$10 and \\$20 usd
* bullet with $x = 4$ math and $x = 5$ more math

$$ x = 2^4 $$

$$ y = 1^3 $$

```python
def example_code():
    return 'hohoho'
```


In [16]:
# see https://stackoverflow.com/questions/11819059/regex-match-character-which-is-not-escaped
s = r'adfsa flaj \abc sdfkla'
r = r'(?<!\\)(?:\\\\)*abc'
re.findall(r, s)

[]

In [17]:
r = r'(?<!\\\\)\$'
s = r'adfsa flaj \\$30 sdfkla $ hoho.'
re.findall(r, s)

['$']

In [40]:
r = r'(?<!\\\\)\$\$.+?\$\$'
s = r'adfsa flaj \\$$30 sdfkla $$x=5$$ hoho.\n$$x = 12$$\nhoho.'
re.findall(r, s)

['$$x=5$$', '$$x = 12$$']

In [76]:
# This regex matches lines in form:   $$some text here$$
# Note: $$ must start and and line
#
#                v-------------- beginning (^): matches begining of input/line
#                |  v----------- pattern (\$\$):   $$ 
#                |  | v--------- dot (.): matches any character except line break
#                |  | |v-------- plus (+): match one or more of preceeding token
#                |  | ||  v----- pattern (\$\$):   $$
#                |  | ||  | v--- end ($): matches end of input/line
#                |vvvv||vvvv| 
pattern_dodo = r'^\$\$.+\$\$$'

def replace_double_dollars(string):
    string_list = list(string)
    for match in re.finditer(pattern_dodo, string, flags=re.M):
        start = match.start()
        end = match.end()
        string_list[start:start+2] = '\['
        string_list[end-2:end] = '\]'
    return ''.join(string_list)

s = 'ala ma\n$$ ALA $$\nkota ala'
r = replace_double_dollars(s)
assert r == 'ala ma\n\[ ALA \]\nkota ala'

In [75]:
r

['$$ ALA $$']

In [70]:
print(s)

ala ma
$$ ALA $$
kota ala


In [71]:
print(replace_double_dollars(s))

ala ma
\[ ALA \]
kota ala


In [72]:
print(source)

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs \\$10 and \\$20 usd
* bullet with $x = 4$ math and $x = 5$ more math

$$ x = 2^4 $$

$$ y = 1^3 $$

```python
def example_code():
    return 'hohoho'
```


In [73]:
print(replace_double_dollars(source))

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs \\$10 and \\$20 usd
* bullet with $x = 4$ math and $x = 5$ more math

\[ x = 2^4 \]

\[ y = 1^3 \]

```python
def example_code():
    return 'hohoho'
```


200

In [ ]:
cell.source.replace('$$', '\[')

In [77]:
tmp = nbformat.v4.new_notebook()

In [78]:
tmp['cells'].append(cell)

In [79]:
html_exporter = nbconvert.HTMLExporter()
html_exporter.template_file = 'basic'

In [80]:
(body, resources) = html_exporter.from_notebook_node(tmp)

In [81]:
print(body)


<div class="cell border-box-sizing text_cell rendered"><div class="prompt input_prompt">
</div><div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>Some good and clear answer goes here</p>
<ul>
<li>maybe a bullet point</li>
<li>or make it two</li>
<li>bullet with $x = 4$ math</li>
</ul>
$$ x = 2^4 $$<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">example_code</span><span class="p">():</span>
    <span class="k">return</span> <span class="s1">&#39;hohoho&#39;</span>
</pre></div>

</div>
</div>
</div>
 


